In [1]:
from my_llm_utils import *

In [2]:
llm = get_llm()

In [3]:
# Check which llm has been loaded
if hasattr(llm, 'model'):
	print(llm.model)  # check loaded model
else:
	print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x10cb48f50> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10cb35950> root_client=<openai.OpenAI object at 0x10c54f550> root_async_client=<openai.AsyncOpenAI object at 0x104704e90> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [4]:
# # test query
# response = llm.invoke("What is the capital of Russia?")
# print(response)

In [5]:
textSample = """Accenture To Train 30,000 Staff On Nvidia AI Tech In Blockbuster Deal

BY DYLAN MARTIN
OCTOBER 2, 2024, 8:00 AM EST
The global services powerhouse says its newly formed Nvidia Business Group will focus on driving enterprise adoption of what it called ‘agentic AI systems’ by taking advantage of key Nvidia software platforms that fuel consumption of GPU-accelerated data centers.


Listen to this article
6 min
429
Shares
linkedin sharing button
twitter sharing button
bluesky sharing button
sharethis sharing button

Global services powerhouse Accenture said it will train 30,000 employees on Nvidia’s full stack of AI technologies and start a new business group dedicated to the AI computing giant as part of a blockbuster deal between the two behemoths.

Accenture, No. 1 on CRN’s 2024 Solution Provider 500 list, announced the expanded partnership on Wednesday and said its newly formed Nvidia Business Group will focus on driving enterprise adoption of what it called “agentic AI systems” by taking advantage of key Nvidia software platforms that fuel consumption of GPU-accelerated data centers.

[Related: These Are Nvidia’s Top 13 Americas Partners With Fast-Growing AI Businesses]

“The power of these announcements cannot be overstated,” Accenture Chief AI Officer Lan Guan said in a briefing with journalists and analysts the day before.

Called the “next frontier” of generative AI, these “agentic AI systems” involve an “army of AI agents” that work alongside human workers to “make decisions and execute with precision across even the most complex workflows,” according to Guan, a 21-year Accenture veteran. Unlike chatbots such as ChatGPT, these agents do not require prompts from humans, and they are not meant to automating pre-existing business steps.

ADVERTISEMENT


How Accenture Plans To Use Nvidia’s Tech

Most of the work done by Accenture’s new business group will happen through its AI Refinery platform, which takes advantage of three key Nvidia software platforms:

Nvidia AI Foundry aids with the development of custom generative AI models by providing access to foundation models, frameworks and tools. Nvidia AI Enterprise offers microservices and tools for developing, optimizing and deploying AI applications. Nvidia Omniverse, on the other hand, enables development and deployment of 3-D applications.

The latter two technologies are paid offerings from Nvidia, and they require systems that are powered by the company’s popular and powerful GPUs.

Julie Sweet, chair and CEO of Accenture, said the company is “breaking significant new ground” and helping clients use generative AI as a catalyst for reinvention.”

“Accenture AI Refinery will create opportunities for companies to reimagine their processes and operations, discover new ways of working, and scale AI solutions across the enterprise to help drive continuous change and create value,” she said in a statement.

In tandem with the launch of the Nvidia business group and massive training effort, Accenture said it’s opening a network of AI engineering hubs in Singapore, Tokyo, Malaga and London that will aid with its push to drive adoption of “agentic AI systems.” These hubs become part of Accenture’s Center for Advanced AI and complement existing locations in Mountain View, Calif., and Bangalore.

Accenture also plans to offer a new microservice meant to enable and optimize virtual facility robot fleet simulation workloads as part of Nvidia’s NIM Agent Blueprints, which is a category of pre-trained, customizable AI workflows that debuted in August.

Why Accenture Sees ‘Agentic AI’ As A Big Deal

Guan called “agentic AI systems” a “game-changer in how our clients reinvent their business” and said these agents will have “self-reflecting capabilities” and a “memory engine” that will allow them to learn from mistakes.

Agents within these systems will specialize in specific tasks, whether that’s to research topics or orchestrate “multiple functions across your enterprise,” according to Guan.

For example, Accenture designed a system consisting of 14 agents that allows its marketing team to “create and run smarter campaigns faster.” The company said this is expected to reduce the manual steps taken for such tasks by 25-35 percent, decrease costs by 6 percent and accelerated completion of said tasks by 22-55 percent.

Elsewhere, the IT services giant said it plans to use its new NIM Agent for robot fleet simulation at Eclipse Automation, an Accenture-owned manufacturing automation company, and it expects the agent to speed up designs by as much as 50 percent.

“I believe that this is the future we are heading towards: AI that does not just respond, but also actually learns, improves and works as a part of the team,” Guan said.

Accenture is doubling down on its partnership with Nvidia after the company said last week that it had $3 billion in new generative AI bookings in its 2024 fiscal year, $1 billion of which came from the fourth quarter that ended on Aug. 31.

Nvidia’s Big Push For Expanded GSI Partnerships

Nvidia’s expanded partnership with Accenture is part of a concerted effort by the AI computing giant to grow its influence with global systems integrators (GSIs) and other multinational IT services companies that have significant influence within enterprises.

Among the other GSIs that have developed extended relationships with Nvidia, Infosys said a year ago that it would train and certify 50,000 employees on the chip designer’s AI hardware and software solutions for example.

At the helm of these efforts is Alvin Da Costa, Nvidia’s former global channel chief who became the leader of its global consulting organization in June 2023. Da Costa told CRN last year that Nvidia formed the new group to focus on its growing relationships with GSIs and service delivery partners—a move backed by Nvidia CEO Jensen Huang.

“Jensen has been a very big advocate of partners, and he’s really giving us the latitude to make our partners profitable, to make them successful,” he said at the time."""

In [6]:
summary = summarize_text(textSample)
print(summary)

Accenture has announced a significant partnership with Nvidia, wherein it will train 30,000 employees on Nvidia's AI technologies and establish a new business group to drive enterprise adoption of "agentic AI systems" using Nvidia's AI platforms, complemented by new AI engineering hubs across multiple global locations.


In [ ]:
import os
import re
import pandas as pd
from typing import Dict, List, Optional, Any
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.parse import urlparse
import langchain
# from langchain_ollama import OllamaLLM
# from langchain_openai import ChatOpenAI
#from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv
import crawl4ai as c4a  # For web scraping

In [ ]:
# State definition for LangGraph
class State:
    data: pd.DataFrame       # The full dataset being processed
    current_index: int       # Tracks which row is currently being processed
    results: Dict[int, Dict] # Stores successful processing results by row index
    errors: List[Dict]       # Collects any errors encountered during processing

    def __init__(self, data):
        self.data = data
        self.current_index = 0
        self.results = {}
        self.errors = []

    def to_dict(self):
        return {
            "current_index": self.current_index,
            "results": self.results,
            "errors": self.errors
        }
